In [1]:
import os
import pandas as pd
import math as m
from time import gmtime, strftime
import numpy as np
import random
import urllib
from fitter import Fitter
import psycopg2
import numpy as np
import cPickle
from collections import defaultdict
import re

In [2]:
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import movie_reviews
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk import tokenize

In [22]:
import pandas as pd # provide sql-like data manipulation tools. very handy.
pd.options.mode.chained_assignment = None
import numpy as np # high dimensional vector computing library.
from copy import deepcopy
from string import punctuation
from random import shuffle

import gensim
from gensim.models.word2vec import Word2Vec # the word2vec model gensim class
LabeledSentence = gensim.models.doc2vec.LabeledSentence # we'll talk about this down below

from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

#from nltk.tokenize import TweetTokenizer # a tweet tokenizer from nltk.
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from keras.layers import Activation, Dense

from keras.models import Sequential

Data Repo

https://archive.ics.uci.edu/ml/datasets/Sentiment+Labelled+Sentences

In [4]:
# Load Dataset : Amazon Product Reviews
data = pd.read_csv('//home//rahul//Downloads//sentiment labelled sentences//amazon_cells_labelled.txt', sep="\t", header = None)
data.columns = ["reviewText", "sentiment"]
print data.shape
data.head()

(1000, 2)


,reviewText,sentiment
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


The format of the SentimenText is not useful. It needs to be tokenized and cleaned.

Here's my tokenizing function that splits each review into tokens.

In [5]:
from nltk.tokenize import RegexpTokenizer
def tokenize(review_text):
    try:
        review_text = unicode(review_text.decode('utf-8').lower())
        Ptokenizer = RegexpTokenizer(r'\w+')
        tokens = Ptokenizer.tokenize(review_text)
        tokens = [word for word in tokens if word not in stopwords.words('english')]
        return tokens
    except:
        return 'NC'

The results of the tokenization should now be cleaned to remove lines with 'NC', resulting from a tokenization error (usually due to weird encoding.)

In [6]:
def postprocess(data, n=1000000):
    data = data.head(n)
    data['tokens'] = data['reviewText'].progress_map(tokenize)  ## progress_map is a variant of the map function plus a progress bar. Handy to monitor DataFrame creations.
    data = data[data.tokens != 'NC']
    data.reset_index(inplace=True)
    data.drop('index', inplace=True, axis=1)
    return data

data = postprocess(data)

progress-bar: 100%|██████████| 1000/1000 [00:01<00:00, 799.32it/s]


The data is now tokenized and cleaned. We are ready to feed it in the word2vec model.

### Building the word2vec model

#### First, let's define a training set and a test set

In [7]:
x_train, x_test, y_train, y_test = train_test_split(np.array(data.head(1000).tokens),\
                                                    np.array(data.head(1000).sentiment), test_size=0.3)

In [8]:
x_train[1:5]

array([list([u'phone', u'works', u'great']),
       list([u'wont', u'work', u'right', u'atleast']),
       list([u'file', u'browser', u'offers', u'options', u'one', u'needs', u'handsfree', u'great']),
       list([u'drain', u'weak', u'snap'])], dtype=object)

In [9]:
y_train[1:5]

array([1, 0, 1, 0])

Before feeding lists of tokens into the word2vec model, we must turn them into LabeledSentence objects beforehand. Here's how to do it:

In [10]:
def labelizeCallText(call_texts, label_type):
    labelized = []
    for i,v in tqdm(enumerate(call_texts)):
        label = '%s_%s'%(label_type,i)
        labelized.append(LabeledSentence(v, [label]))
    return labelized

x_train = labelizeCallText(x_train, 'TRAIN')
x_test = labelizeCallText(x_test, 'TEST')

700it [00:00, 246206.52it/s]
300it [00:00, 88862.37it/s]


Let's check the first element from x_train.

In [11]:
x_train[0]

TaggedDocument(words=[u'none', u'tones', u'acceptable'], tags=['TRAIN_0'])

Ok so each element is basically some object with two attributes: a list (of tokens) and a label.
Now we are ready to build the word2vec model from x_train i.e. the corpus.

In [12]:
review_text_w2v = Word2Vec(size=200, min_count=10)
review_text_w2v.build_vocab([x.words for x in tqdm(x_train)])
review_text_w2v.train([x.words for x in tqdm(x_train)],total_examples=len(x_train), epochs=10)

100%|██████████| 700/700 [00:00<00:00, 1272654.01it/s]


3896

The code is self-explanatory.
* On the first line the model is initialized with the dimension of the vector space (we set it to 200) and min_count (a threshold for filtering words that appear less)
* On the second line the vocabulary is created.
* On the third line the model is trained i.e. its weights are updated.

Once the model is built and trained on the corpus of call texts, we can use it to convert words to vectors. Here's an example:

In [13]:
review_text_w2v['good']

array([ -5.89024276e-04,   1.90061599e-03,   2.71252589e-04,
        -5.41494228e-04,  -1.86419720e-03,   9.95092909e-04,
        -1.61273812e-03,  -2.42068805e-03,   1.86959805e-03,
        -1.79923096e-04,   1.39681890e-03,   2.29077390e-03,
         1.17142117e-04,  -1.11752561e-04,  -2.25103134e-03,
        -8.53066391e-04,  -9.32306226e-04,   1.45286089e-03,
         4.81377152e-04,   1.78067293e-03,   1.51930714e-03,
        -2.27694702e-03,   2.47723539e-03,   1.33213797e-03,
        -2.21236981e-03,   2.24324293e-03,   9.10254585e-06,
        -7.18741561e-04,  -1.87078875e-03,  -4.00763856e-05,
        -1.05708803e-03,  -2.02402729e-03,   9.44534724e-04,
         1.32431998e-03,   1.84042682e-03,   5.91372489e-04,
         9.42600949e-04,  -1.74512935e-03,  -1.32808601e-03,
        -1.38255360e-03,   1.10342167e-04,  -2.11775349e-03,
        -1.81310414e-03,   1.66405435e-03,   1.68973929e-03,
        -2.26604869e-03,   7.97806249e-04,   2.42115231e-03,
        -1.13030977e-03,

You can check: this is a 200-dimension vector. Of course, we can only get the vectors of the words of the corpus.
Let's try something else. We spoke earlier about semantic relationships. Well, the Word2Vec gensim implementation provides a cool method named most_similar. Given a word, this method returns the top n similar ones. This is an interesting feature. Let's try it on some words:

In [14]:
review_text_w2v.most_similar('good')

[(u'also', 0.2034391462802887),
 (u'one', 0.17719045281410217),
 (u'time', 0.16582369804382324),
 (u'work', 0.15659163892269135),
 (u'great', 0.11894215643405914),
 (u'excellent', 0.1180928573012352),
 (u'fit', 0.11151019483804703),
 (u'2', 0.11087364703416824),
 (u'really', 0.10830893367528915),
 (u'ever', 0.10527921468019485)]

How awesome is that?

For a given word, we get similar surrounding words of same context. Basically these words have a probability to be closer to that given word in most of the call texts.

It's interesting to see that our model gets facebook, twitter, skype together and bar, restaurant and cafe together as well. This could be useful for building a knowledge graph. Any thoughts about that?

How about visualizing these word vectors? We first have to reduce their dimension to 2 using t-SNE. Then, using an interactive visualization tool such as Bokeh, we can map them directly on 2D plane and interact with them.


### Building a sentiment classifier

Let's now get to the sentiment classification part. As for now, we have a word2vec model that converts each word from the corpus into a high dimensional vector. This seems to work fine according to the similarity tests.

In order to classify call texts, we have to turn them into vectors as well. How could we do this? Well, this task is almost done. Since we know the vector representation of each word composing a call texts, we have to "combine" these vectors together and get a new one that represents the reviews as a whole.

A first approach consists in averaging the word vectors together. But a slightly better solution I found was to compute a weighted average where each weight gives the importance of the word with respect to the corpus. Such a weight could the tf-idf score. To learn more about tf-idf, you can look at <> article.

Let's start by building a tf-idf matrix.

In [15]:
print 'building tf-idf matrix ...'
vectorizer = TfidfVectorizer(analyzer=lambda x: x, min_df=10)
matrix = vectorizer.fit_transform([x.words for x in x_train])
tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
print 'vocab size :', len(tfidf)

building tf-idf matrix ...
vocab size : 62


Now let's define a function that, given a list of call text tokens, creates an averaged review vector.

In [16]:
def buildWordVector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += review_text_w2v[word].reshape((1, size)) * tfidf[word]
            count += 1.
        except KeyError: # handling the case where the token is not
                         # in the corpus. useful for testing.
            continue
    if count != 0:
        vec /= count
    return vec

Now we convert x_train and and x_test into list of vectors using this function. We also scale each column to have zero mean and unit standard deviation.

In [17]:
from sklearn.preprocessing import scale
train_vecs_w2v = np.concatenate([buildWordVector(z, 200) for z in tqdm(map(lambda x: x.words, x_train))])
train_vecs_w2v = scale(train_vecs_w2v)

test_vecs_w2v = np.concatenate([buildWordVector(z, 200) for z in tqdm(map(lambda x: x.words, x_test))])
test_vecs_w2v = scale(test_vecs_w2v)

100%|██████████| 300/300 [00:00<00:00, 26636.14it/s]


We should now be ready to feed these vectors into a neural network classifier. In fact, using Keras is very easy to define layers and activation functions.
Here is a basic 2-layer architecture.

In [18]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=200))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

model.fit(train_vecs_w2v, y_train,  batch_size=32, verbose=2)

Epoch 1/10
0s - loss: 0.7215 - acc: 0.5900
Epoch 2/10
0s - loss: 0.5685 - acc: 0.7186
Epoch 3/10
0s - loss: 0.5230 - acc: 0.7414
Epoch 4/10
0s - loss: 0.4975 - acc: 0.7700
Epoch 5/10
0s - loss: 0.4774 - acc: 0.7829
Epoch 6/10
0s - loss: 0.4641 - acc: 0.7900
Epoch 7/10
0s - loss: 0.4536 - acc: 0.7986
Epoch 8/10
0s - loss: 0.4479 - acc: 0.7886
Epoch 9/10
0s - loss: 0.4327 - acc: 0.8014
Epoch 10/10
0s - loss: 0.4283 - acc: 0.7943


In [19]:
import tensorflow as tf


print tf.__version__

1.1.0


Now that the model is trained, let's evaluate it on the test set:

In [20]:
score = model.evaluate(test_vecs_w2v, y_test, batch_size=128, verbose=2)
print score[1]

0.720000003179


In [21]:
score

[0.52593540668487548, 0.72000000317891444]

72% accuracy in the first swing of the model is not bad. We could eventually tune more parameters in the word2vec model and the neural network classifer to reach a higher precision score. 

Please tell me if you managed to do so.